# Training a Transformer Model on Custom Data

This notebook demonstrates how to set up, configure, and train a Transformer model using a custom dataset.

In [ ]:
# Clone the GitHub repository
!git clone https://github.com/ibrardu/llm-transformer.git

# Navigate to the cloned repository
%cd llm-transformer

# Install the required dependencies
!pip install -r requirements.txt

# Create an empty __init__.py file in the models directory
!touch models/__init__.py


## Load and Preprocess Data

In [ ]:
import os
import torch
import pandas as pd

# Load your dataset
data_path = 'data/your_dataset.txt'  # Modify this line with your actual dataset path
with open(data_path, 'r', encoding='utf-8') as f:
    text = f.read()

# Preprocess the dataset
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))  # 90% for training, rest for validation
train_data = data[:n]
val_data = data[n:]

print(f"Vocabulary size: {vocab_size}")
print(f"Training data size: {len(train_data)}")
print(f"Validation data size: {len(val_data)}")


## Define the Model

In [ ]:
import torch.nn as nn
from models.bigram_model import BigramLanguageModel

# Configuration
n_layer = 6
n_head = 4
n_embd = 256
block_size = 64
dropout = 0.0
learning_rate = 1e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Initialize model
model = BigramLanguageModel(vocab_size, n_embd, n_head, n_layer, block_size, dropout)
model.to(device)

# Print the number of parameters
print(f"Model has {sum(p.numel() for p in model.parameters() if p.requires_grad)} trainable parameters")

## Training Setup

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
batch_size = 16
eval_interval = 100
eval_iters = 200
max_iters = 10000

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

## Train the Model

In [ ]:
for iter in range(max_iters):
    # Evaluate loss on train and val sets every eval_interval
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    
    # Sample a batch of data
    xb, yb = get_batch('train')
    
    # Evaluate the loss and perform backpropagation
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

## Generate Text

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_text = decode(model.generate(context, max_new_tokens=2000)[0].tolist())
print(generated_text)